# Introduction
This notebook contains answers to the Fetch Rewards Coding Exercise.

# Scenario
I have been given three JSON tables and will answer questions posed by a business stakeholder. For this exercise, I used SQLite and Python to answer the following questions.

# Q1) Review Existing Unstructured Data and Diagram a New Structured Relational Data Model


To begin this part of the exercise, I first need to see what the data looks like, convert the data from a JSON object into a pandas dataframe and then create the final diagram using LucidChart. The diagram will be found as a file in this Github repo.

In [1]:
import pandas as pd
import json

In [2]:
## user data
json_to_lst = []
with open('users.json','r') as f:
    json_data = f.readlines()
    for json_line in json_data:
        json_obj = json.loads(json_line)
        user_id = json_obj['_id']['$oid']
        active_user = json_obj['active']
        created_date = json_obj['createdDate']['$date']
        last_login = json_obj['lastLogin']['$date'] if 'lastLogin' in json_obj else None
        role = json_obj['role']
        signup_source = json_obj['signUpSource'] if 'signUpSource' in json_obj else None
        state = json_obj['state'] if 'state' in json_obj else None
        json_to_lst.append([user_id, active_user, created_date, last_login, role, signup_source, state])

In [3]:
users_df = pd.DataFrame(json_to_lst, columns=['user_id', 'user_active', 'created_date', 'last_login', 'role', 'signup_source', 'state'])
users_df.head()

,user_id,user_active,created_date,last_login,role,signup_source,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [4]:
## receipts
receipts_lst = []
with open('receipts.json', 'r') as f:
    receipts_json = f.readlines()
    for line in receipts_json:
        receipts_obj = json.loads(line)
        receipt_id = receipts_obj['_id']['$oid']
        bonus_pts = receipts_obj['bonusPointsEarned'] if 'bonusPointsEarned' in receipts_obj else None
        pts_reason = receipts_obj['bonusPointsEarnedReason'] if 'bonusPointsEarnedReason' in receipts_obj else None
        receipt_date = receipts_obj['createDate']['$date']
        date_scanned = receipts_obj['dateScanned']['$date']
        finished_date = receipts_obj['finishedDate']['$date'] if 'finishedDate' in receipts_obj else None
        modify_date = receipts_obj['modifyDate']['$date']
        award_date = receipts_obj['pointsAwardedDate']['$date'] if 'pointsAwardedDate' in receipts_obj else None
        pts_earned = receipts_obj['pointsEarned'] if 'pointsEarned' in receipts_obj else None
        purchase_date = receipts_obj['purchaseDate']['$date'] if 'purchaseDate' in receipts_obj else None
        item_count = receipts_obj['purchasedItemCount'] if 'purchasedItemCount' in receipts_obj else None
        reward_status = receipts_obj['rewardsReceiptStatus']
        tot_spent = receipts_obj['totalSpent'] if 'totalSpent' in receipts_obj else None
        receipt_user_id = receipts_obj['userId']
        receipts_lst.append([receipt_id, bonus_pts, pts_reason, date_scanned, finished_date, modify_date, award_date, pts_earned, purchase_date, item_count, reward_status, tot_spent, receipt_user_id])

In [5]:
receipts_df = pd.DataFrame(receipts_lst, columns=['receipt_id', 'bonus_pts', 'pts_reason', 'date_scanned', 'finished_date', 'modify_date', 'award_date', 'pts_earned', 'purchase_date', 'item_count','reward_status', 'tot_spent', 'receipt_user_id'])
receipts_df.head()

,receipt_id,bonus_pts,pts_reason,date_scanned,finished_date,modify_date,award_date,pts_earned,purchase_date,item_count,reward_status,tot_spent,receipt_user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,NaN,1609687542000,NaN,5,1.609632e+12,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


Inspecting the receipt JSON object, I realized that the receiptsItemList had a nested dictionary enclosed in a list. Because of this, I will separate the column from the main receipts table and make it it's own table as seen below. The two tables will be connected using the receipt_id key.

In [6]:
## receiptsItemList
items_lst = []
with open('receipts.json', 'r') as f:
    receipts_json = f.readlines()
    for line in receipts_json:
        receipt_items_obj = json.loads(line)
        receipt_id = receipt_items_obj['_id']['$oid']
        items_on_receipt = receipt_items_obj.get('rewardsReceiptItemList')
        if items_on_receipt is not None:
            for items in items_on_receipt:
                barcode = items.get('barcode')
                description = items.get('description')
                final_price = items.get('finalPrice')
                item_price = items.get('itemPrice')
                need_review = items.get('needsFetchReview')
                partner_item_id = items.get('partnerItemId')
                quantity = items.get('quantityPurchased')
                flagged_barcode = items.get('userFlaggedBarcode')
                new_item = items.get('userFlaggedNewItem')
                flagged_price = items.get('userFlaggedPrice')
                flagged_quantity = items.get('userFlaggedQuantity')
                items_lst.append([receipt_id,barcode, description, final_price, item_price, need_review, partner_item_id, quantity, flagged_barcode, new_item, flagged_price, flagged_quantity])

In [7]:
items_df = pd.DataFrame(items_lst, columns=['receipt_id','barcode','description', 'final_price', 'item_price', 'need_review', 'partner_item_id', 'quantity', 'flagged_barcode', 'new_item', 'flagged_price', 'flagged_quantity'])
items_df.head()

,receipt_id,barcode,description,final_price,item_price,need_review,partner_item_id,quantity,flagged_barcode,new_item,flagged_price,flagged_quantity
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,None,1,1.0,None,None,None,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,1.0,028400642255,True,10.00,1.0
3,5ff1e1f10a720f052300057a,None,None,None,None,False,1,NaN,4011,True,26.00,3.0
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,4.0,4011,True,28.00,4.0


In [8]:
##brands
brands_lst = []
with open('brands.json', 'r') as f:
    brands_json = f.readlines()
    for brands in brands_json:
        brands_obj = json.loads(brands)
        brand_id = brands_obj['_id']['$oid']
        barcode = brands_obj['barcode']
        brand_code = brands_obj['brandCode'] if 'brandCode' in brands_obj else None
        cat = brands_obj['category'] if 'category' in brands_obj else None
        cat_code = brands_obj['categoryCode'] if 'categoryCode' in brands_obj else None
        cpg = brands_obj['cpg']['$id']['$oid']
        brand_name = brands_obj['name']
        top_brand = brands_obj['topBrand'] if 'topBrand' in brands_obj else None
        brands_lst.append([brand_id, barcode, brand_code, cat, cat_code,cpg, brand_name, top_brand])

In [9]:
brands_df = pd.DataFrame(brands_lst, columns=['brand_id', 'barcode', 'brand_code', 'cat', 'cat_code', 'cpg', 'brand_name', 'top_brand'])
brands_df.head()

,brand_id,barcode,brand_code,cat,cat_code,cpg,brand_name,top_brand
0,601ac115be37ce2ead437551,511111019862,None,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,False


In [10]:
##creating csv files
brands_df.to_csv('brands.csv', index = False)
items_df.to_csv('items.csv', index = False)
receipts_df.to_csv('receipts.csv', index = False)
users_df.to_csv('users.csv', index = False)

In conclusion, there are four tables in the diagram. The receipts and receipt item list tables are connected via the receipt_id column(primary key for the receipts table and foriegn key for the items list table). The receipts and users table are connected via the user_id/receipt_user_id columns. And the brands and item list tables are connected via the barcode column.

# Q2) Write a query that answers a predetermined question from a business stakeholder

## When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [11]:
receipts_df.columns

Index(['receipt_id', 'bonus_pts', 'pts_reason', 'date_scanned',
       'finished_date', 'modify_date', 'award_date', 'pts_earned',
       'purchase_date', 'item_count', 'reward_status', 'tot_spent',
       'receipt_user_id'],
      dtype='object')

In [12]:
import sqlite3
conn = sqlite3.connect(":memory:")
c = conn.cursor()
c.execute('CREATE TABLE receipts (receipt_id char, bonus_pts int, pts_reason varchar, date_scanned int, finished_date float, modify_date int, award_date int, pts_earned int, purchase_date int, item_count int, reward_status char, tot_spent float, receipt_user_id int)')
receipts = pd.read_csv('receipts.csv')
receipts.to_sql('receipts', conn, if_exists='append', index=False)

In [13]:
receipts['reward_status'].value_counts()

FINISHED     518
SUBMITTED    434
REJECTED      71
PENDING       50
FLAGGED       46
Name: reward_status, dtype: int64

Below I made the assumption that FINISHED is in place of ACCEPTED.

In [14]:
avg_spent_sql = '''
SELECT reward_status, SUM(item_count) AS tot_count, AVG(tot_spent) AS avg_tot_spent
FROM receipts 
WHERE reward_status = 'FINISHED' OR reward_status = 'REJECTED' 
GROUP BY reward_status'''

pd.read_sql(avg_spent_sql, conn)

,reward_status,tot_count,avg_tot_spent
0,FINISHED,8184,80.854305
1,REJECTED,173,23.326056


As we can see, the average amount spent and the total amount of items  is greated when the reward status is finished. The average amount spent when reward status is finished is 80.86 dollars and the total count is 8184.

# Q3) Evaluate Data Quality Issues in the Data Provided


For this part of the exercise, I will be using python to evaluate the quality of the data. The first thing I will do in this section is check the dataframes.

In [15]:
users_df.head()

,user_id,user_active,created_date,last_login,role,signup_source,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [16]:
receipts_df.head()

,receipt_id,bonus_pts,pts_reason,date_scanned,finished_date,modify_date,award_date,pts_earned,purchase_date,item_count,reward_status,tot_spent,receipt_user_id
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,NaN,1609687542000,NaN,5,1.609632e+12,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052


In [17]:
brands_df.head()

,brand_id,barcode,brand_code,cat,cat_code,cpg,brand_name,top_brand
0,601ac115be37ce2ead437551,511111019862,None,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,False
1,601c5460be37ce2ead43755f,511111519928,STARBUCKS,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,False
2,601ac142be37ce2ead43755d,511111819905,TEST BRANDCODE @1612366146176,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,False
3,601ac142be37ce2ead43755a,511111519874,TEST BRANDCODE @1612366146051,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146051,False
4,601ac142be37ce2ead43755e,511111319917,TEST BRANDCODE @1612366146827,Candy & Sweets,CANDY_AND_SWEETS,5332fa12e4b03c9a25efd1e7,test brand @1612366146827,False


In [18]:
items_df.head()

,receipt_id,barcode,description,final_price,item_price,need_review,partner_item_id,quantity,flagged_barcode,new_item,flagged_price,flagged_quantity
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,5.0,4011,True,26.00,5.0
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,None,1,1.0,None,None,None,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,1.0,028400642255,True,10.00,1.0
3,5ff1e1f10a720f052300057a,None,None,None,None,False,1,NaN,4011,True,26.00,3.0
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,4.0,4011,True,28.00,4.0


A brief look at the data, I can see that there are some missing values and the date columns in the dataframes are inputted in a strange format, that does not look like datetimes or timestamp values. Now I will look for the amount of missing values I am dealing with.

In [19]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   receipt_id       1119 non-null   object 
 1   bonus_pts        544 non-null    float64
 2   pts_reason       544 non-null    object 
 3   date_scanned     1119 non-null   int64  
 4   finished_date    568 non-null    float64
 5   modify_date      1119 non-null   int64  
 6   award_date       537 non-null    float64
 7   pts_earned       609 non-null    float64
 8   purchase_date    671 non-null    float64
 9   item_count       635 non-null    float64
 10  reward_status    1119 non-null   object 
 11  tot_spent        684 non-null    float64
 12  receipt_user_id  1119 non-null   object 
dtypes: float64(7), int64(2), object(4)
memory usage: 113.8+ KB


In [20]:
items_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   receipt_id        6941 non-null   object 
 1   barcode           3090 non-null   object 
 2   description       6560 non-null   object 
 3   final_price       6767 non-null   object 
 4   item_price        6767 non-null   object 
 5   need_review       813 non-null    object 
 6   partner_item_id   6941 non-null   object 
 7   quantity          6767 non-null   float64
 8   flagged_barcode   337 non-null    object 
 9   new_item          323 non-null    object 
 10  flagged_price     299 non-null    object 
 11  flagged_quantity  299 non-null    float64
dtypes: float64(2), object(10)
memory usage: 650.8+ KB


In [21]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   user_id        495 non-null    object 
 1   user_active    495 non-null    bool   
 2   created_date   495 non-null    int64  
 3   last_login     433 non-null    float64
 4   role           495 non-null    object 
 5   signup_source  447 non-null    object 
 6   state          439 non-null    object 
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [22]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   brand_id    1167 non-null   object
 1   barcode     1167 non-null   object
 2   brand_code  933 non-null    object
 3   cat         1012 non-null   object
 4   cat_code    517 non-null    object
 5   cpg         1167 non-null   object
 6   brand_name  1167 non-null   object
 7   top_brand   555 non-null    object
dtypes: object(8)
memory usage: 73.1+ KB


Inspecting the four tables, I can see that there are some missing/null values. I will now inspect that closely.

In [23]:
receipts.isna().sum()/receipts.shape[0]

receipt_id         0.000000
bonus_pts          0.513852
pts_reason         0.513852
date_scanned       0.000000
finished_date      0.492404
modify_date        0.000000
award_date         0.520107
pts_earned         0.455764
purchase_date      0.400357
item_count         0.432529
reward_status      0.000000
tot_spent          0.388740
receipt_user_id    0.000000
dtype: float64

In [24]:
items_df.isna().sum()/items_df.shape[0]

receipt_id          0.000000
barcode             0.554819
description         0.054891
final_price         0.025068
item_price          0.025068
need_review         0.882870
partner_item_id     0.000000
quantity            0.025068
flagged_barcode     0.951448
new_item            0.953465
flagged_price       0.956923
flagged_quantity    0.956923
dtype: float64

In [25]:
brands_df.isna().sum()/brands_df.shape[0]

brand_id      0.000000
barcode       0.000000
brand_code    0.200514
cat           0.132819
cat_code      0.556984
cpg           0.000000
brand_name    0.000000
top_brand     0.524422
dtype: float64

In [26]:
users_df.isna().sum()/users_df.shape[0]

user_id          0.000000
user_active      0.000000
created_date     0.000000
last_login       0.125253
role             0.000000
signup_source    0.096970
state            0.113131
dtype: float64

Looking at the ratios of missing values, one thing that is problematic is that about 55% of the barcode data in the items table is missing. This would make it difficult to connect many of the items to their brand in the brands table. 

# Q4) Communicate with Stakeholders

Hi there,<br>
<br>
I was doing a general data quality check and found that over 55% of receipt items are missing a barcode.
Is it normal to have so many receipt items without an unique barcode? These barcodes are useful because they tell us which brand an item belongs to. If this percentage of missing barcodes is not normal, we would need to fix the data pipeline that loads this data into our data warehouse. To make sure this problem has been fixed, it would be good to know the percentage of receipt items that should be missing a barcode. I am not that familiar with our data pipelines, so I could imagine that it might be a challenge to get this data. Please let me know if that is the case.<br>

Best Regards,<br>
Hazel Donaldson<br>